<a href="https://colab.research.google.com/github/Cody9494/LEGALSKEPSIS-DATA/blob/main/LAWSKPEPSIS_STEP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Φόρτωση του dataset με τα embeddings
df_chunks = pd.read_parquet("/content/drive/MyDrive/LAWSKEPSIS/df_chunks_with_embeddings.parquet")

# Εξασφάλιση ότι τα embeddings είναι tensor
df_chunks["embedding"] = df_chunks["embedding"].apply(lambda x: torch.tensor(x))


In [4]:
model = SentenceTransformer("intfloat/e5-mistral-7b-instruct")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [5]:
df_chunks

,act_id,chunk_index,chunk_text,token_len,embedding
0,2023/207,0,Title: (EU) 2023/207 Commission Delegated Regu...,240,"[tensor(2.7614), tensor(6.6488), tensor(-0.977..."
1,2025/1003,0,Title: (EU) 2025/1003 Commission Delegated Reg...,1000,"[tensor(-0.6529), tensor(4.8847), tensor(-1.72..."
2,2025/1003,1,participants and competent authorities to dete...,471,"[tensor(-4.0273), tensor(3.1027), tensor(-1.27..."
3,2016/182,0,Title: (EU) 2016/182 Commission Implementing R...,786,"[tensor(2.9411), tensor(1.8080), tensor(-1.645..."
4,2010/222,0,Title: Commission Decision of 19 April 2010 20...,319,"[tensor(-0.2327), tensor(0.8258), tensor(-0.72..."
...,...,...,...,...,...
62243,2008/170,0,Title: Commission Decision of 27 February 2008...,775,"[tensor(2.6734), tensor(3.2488), tensor(-2.550..."
62244,2014/142,0,Title: 2014/142/CFSP Council Implementing Deci...,164,"[tensor(3.8137), tensor(3.0113), tensor(-1.689..."
62245,2023/2821,0,Title: DRAFT DECISION OF THE EEA JOINT COMMITT...,208,"[tensor(3.9866), tensor(4.4613), tensor(-2.899..."
62246,2023/2821,0,Title: (EU) 2023/2821 Council Decision of 4 De...,346,"[tensor(2.9014), tensor(5.4077), tensor(-3.819..."


In [26]:
def semantic_search_grouped_by_act(query, model, df_chunks, top_k=5, aggregation="max"):
    # 1. Ενσωμάτωση ερωτήματος
    query_emb = model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

    # 2. Πίνακας embeddings
    all_embeddings = torch.stack(df_chunks["embedding"].to_list()).to(query_emb.device)

    # 3. Υπολογισμός cosine similarity
    cosine_scores = F.cosine_similarity(query_emb.unsqueeze(0), all_embeddings)

    df_chunks = df_chunks.copy()
    df_chunks["similarity"] = cosine_scores.cpu().numpy()

    # 4. Ομαδοποίηση ανά act_id με aggregation
    if aggregation == "max":
        act_scores = df_chunks.groupby("act_id")["similarity"].max().reset_index()
    elif aggregation == "mean":
        act_scores = df_chunks.groupby("act_id")["similarity"].mean().reset_index()
    elif aggregation == "top3mean":
        act_scores = (
            df_chunks.sort_values("similarity", ascending=False)
            .groupby("act_id")
            .head(3)
            .groupby("act_id")["similarity"].mean()
            .reset_index()
        )
    else:
        raise ValueError("Unknown aggregation method")

    # 5. Επιλογή top-k act_id
    top_acts = act_scores.sort_values("similarity", ascending=False).head(top_k)

    # 6. Ανάκτηση κορυφαίου chunk ανά act
    result_chunks = (
        df_chunks[df_chunks["act_id"].isin(top_acts["act_id"])]
        .sort_values(["act_id", "similarity"], ascending=[True, False])
        .groupby("act_id")
        .head(1)
    )

    return result_chunks[["act_id", "chunk_index", "chunk_text", "similarity"]].reset_index(drop=True)


In [14]:
# 2. Αφαίρεση chunks με λίγους tokens
df_filtered = df_chunks[df_chunks["token_len"] > 30]


# 3. Θορυβώδεις λέξεις-κλειδιά
noise_keywords = ["http", "www.", "europa.eu", "italic", "ELI:", "OJ L", "data.europa.eu", "screen/expert-groups"]

# 4. Υπολογισμός ποσοστού θορύβου ανά chunk
def noise_ratio(text, noise_keywords):
    words = text.split()
    if not words:
        return 1.0
    noise_count = sum(any(kw.lower() in word.lower() for kw in noise_keywords) for word in words)
    return noise_count / len(words)

df_filtered["noise_ratio"] = df_filtered["chunk_text"].apply(lambda txt: noise_ratio(txt, noise_keywords))

# 5. Φιλτράρισμα chunks με >30% θορυβώδεις λέξεις
df_filtered = df_filtered[df_filtered["noise_ratio"] < 0.3].reset_index(drop=True)

# 6. Προβολή δείγματος
import random
sample = df_filtered.sample(3, random_state=42)
pd.set_option("display.max_colwidth", None)
display(sample[["act_id", "chunk_index", "chunk_text", "token_len", "noise_ratio"]])

/tmp/ipython-input-2096607421.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["noise_ratio"] = df_filtered["chunk_text"].apply(lambda txt: noise_ratio(txt, noise_keywords))


act_id  chunk_index  \
12184   1993/550            0   
12727   2025/670            5   
18622  2021/1341            1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [15]:
len(df_filtered)

61762

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-mistral-7b-instruct")


In [20]:
df_filtered

act_id  chunk_index  \
0       2023/207            0   
1      2025/1003            0   
2      2025/1003            1   
3       2016/182            0   
4       2010/222            0   
...          ...          ...   
61757   2008/170            0   
61758   2014/142            0   
61759  2023/2821            0   
61760  2023/2821            0   
61761  2017/1372            0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [27]:
query = "Can the EU freeze a person’s assets as part of sanctions?"

top_acts = semantic_search_grouped_by_act(
    query,
    model=model,
    df_chunks=df_filtered,  # το noise-filtered df
    top_k=5,
    aggregation="max",  # ή "mean", "top3mean"
)

pd.set_option('display.max_colwidth', None)
display(top_acts)


act_id  chunk_index  \
0  2023/2871            1   
1  2024/2163           15   
2  2024/2465            2   
3  2025/1485            2   
4  2025/2032            2   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       